# Example of using simsopt to optimize simsgeo objects

This notebook demonstrates a silly little example of optimizing a simsgeo curve using simsopt. We start with a curve that has random Fourier coefficients for $R(\phi)$ and $Z(\phi)$, where $(R, \phi, Z)$ are cylindrical coordinates. We minimize the length of the curve, keeping the 0-frequency Fourier mode of $R$ fixed. The result should be a circle.

As of now, one change is needed in simsgeo to get this to work: in `simsgeo/simsgeo/magneticaxis.py`, `set_dofs()` should call `self.invalidate_cache()`.

In [1]:
from simsgeo import StelleratorSymmetricCylindricalFourierCurve, CurveLength
import numpy as np
import sys
sys.path.append('..')
from simsopt import optimizable, LeastSquaresTerm, LeastSquaresProblem

Initialize the simsgeo curve. Simsopt's `optimizable` function adds methods for holding degrees of freedom fixed.

In [2]:
nquadrature = 100
nfourier = 4
nfp = 5
curve = optimizable(StelleratorSymmetricCylindricalFourierCurve(nquadrature, nfourier, nfp))

Initialize the Fourier coefficients to random numbers. The first coefficient is the average major radius, so set it to a positive number.

In [4]:
x0 = np.random.rand(curve.num_dofs()) - 0.5
x0[0] = 3.0
curve.set_dofs(x0)
print('Initial curve dofs: ', curve.get_dofs())

Initial curve dofs:  [ 3.          0.03934276 -0.41214021 -0.34998505 -0.1027793   0.04387313
 -0.45589641 -0.09285888  0.01371765]


Tell the curve object that the first Fourier mode is fixed, whereas all the other dofs are not.

In [5]:
curve.all_fixed(False)
curve.fixed[0] = True

Presently in simsgeo, the length objective is a separate object rather than as a function of Curve itself, so we must create this 2nd object:

In [7]:
obj = optimizable(CurveLength(curve))
print('Initial curve length: ', obj.J())

Initial curve length:  39.28955832805552


The following assignment is needed for simsopt to recognize that `obj` depends on `curve`. Hopefully this step can be made unnecessary with a change in simsgeo.

In [8]:
obj.depends_on = ['curve']

Create a term in the least-squares objective function, $1.0 * (length - 0.0)^2$:

In [9]:
term1 = LeastSquaresTerm(obj, 0.0, 1.0)

Put this term into a least-squares problem object:

In [10]:
prob = LeastSquaresProblem([term1])

Simsopt recognizes that derivative information is available:

In [11]:
prob.dofs.grad_avail

True

Let's view the initial global state vector. It should contain all the initial Fourier modes except the first:

In [13]:
prob.x

array([ 0.03934276, -0.41214021, -0.34998505, -0.1027793 ,  0.04387313,
       -0.45589641, -0.09285888,  0.01371765])

Solve the minimization problem:

In [14]:
prob.solve()

Using analytic derivatives
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         7.7183e+02                                    1.08e+03    
       1              2         2.3574e+02      5.36e+02       7.23e-01       5.52e+02    
       2              4         1.8754e+02      4.82e+01       1.81e-01       2.69e+02    
       3              6         1.7999e+02      7.55e+00       4.52e-02       8.43e+01    
       4              7         1.7823e+02      1.76e+00       4.52e-02       7.51e+01    
       5              9         1.7772e+02      5.07e-01       1.13e-02       1.35e+01    
       6             11         1.7768e+02      3.65e-02       2.83e-03       5.85e+00    
       7             12         1.7767e+02      1.59e-02       2.83e-03       2.69e+00    
       8             13         1.7766e+02      6.45e-03       2.83e-03       4.27e+00    
       9             15         1.7766e+02      2.77e-03       

Examine properties of the optimum found:

In [15]:
print('At the optimum, x: ', prob.x)
print(' Final curve dofs: ', curve.get_dofs())
print(' Final curve length:    ', obj.J())
print(' Expected final length: ', 2 * np.pi * x0[0])
print(' objective function: ', prob.objective)

At the optimum, x:  [ 1.06023635e-04  1.81287904e-09  4.21471514e-10 -9.27440562e-06
 -1.11663809e-04 -1.86116878e-09 -1.65496223e-10  1.74263584e-06]
 Final curve dofs:  [ 3.00000000e+00  1.06023635e-04  1.81287904e-09  4.21471514e-10
 -9.27440562e-06 -1.11663809e-04 -1.86116878e-09 -1.65496223e-10
  1.74263584e-06]
 Final curve length:     18.849556250550418
 Expected final length:  18.84955592153876
 objective function:  355.3057708426643


All the Fourier modes after the first (the 0-frequency mode) have been reduced towards 0. The final curve length is approximately that of the circle with the expected radius.